# Probabilistic Modeling with Bayes Net

In this assignment, you will work with probabilistic models known as Bayesian Networks to efficiently calculate the answer to probability questions concerning discrete random variables.

In [ ]:
from utils.bayes_net import BayesNet, Node
import utils.vis as vis

## Part 1: Design Bayesian Network

To start, design a basic probabilistic model for the following system:

There's a nuclear power plant in which an alarm is supposed to ring when the core temperature, indicated by a gauge, exceeds a fixed threshold. For simplicity, we assume that the temperature is represented as either high or normal. However, the alarm is sometimes faulty, and the gauge is more likely to fail when the temperature is high. Use the following Boolean variables in your implementation:

- A = alarm sounds
- F<sub>A</sub> = alarm is faulty
- G = gauge reading (high = True, normal = False)
- F<sub>G</sub> = gauge is faulty
- T = actual temperature (high = True, normal = False)

Assume that the following statements about the system are true:

1. The temperature gauge reads the correct temperature with 95% probability when it is not faulty and 20% probability when it is faulty. For simplicity, say that the gauge's "true" value corresponds with its "hot" reading and "false" with its "normal" reading, so the gauge would have a 95% chance of returning "true" when the temperature is hot and it is not faulty.
2. The alarm is faulty 15% of the time.
3. The temperature is hot (call this "true") 20% of the time.
4. When the temperature is hot, the gauge is faulty 80% of the time. Otherwise, the gauge is faulty 5% of the time.
5. The alarm responds correctly to the gauge 55% of the time when the alarm is faulty, and it responds correctly to the gauge 90% of the time when the alarm is not faulty. For instance, when it is faulty, the alarm sounds 55% of the time that the gauge is "hot" and remains silent 55% of the time that the gauge is "normal."

Design a bayesian network for this system and set the conditional probabilities for the necessary variables on the network using utils functions and classes to represent the nodes and conditional probability arcs connecting nodes.

The following cell is an example of designing a simple bayes net using utils:

In [ ]:
# Nodes
A = Node("A", [0, 1])
C = Node("C", [0, 1])
B = Node("B", [0, 1])

B.set_parents([A, C])

A.set_cpt([0.7, 0.3])
C.set_cpt([0.6, 0.4])
B.set_cpt([
    [[0.9, 0.4], [0.3, 0.1]], # P(B=0|A,C)
    [[0.1, 0.6], [0.7, 0.9]]  # P(B=1|A,C)
])

bn = BayesNet([A, B, C])

vis.plot_bayes_net(bn)
vis.plot_all_cpt_tables(bn)

In [ ]:
def make_power_plant_net():
    # Create nodes with boolean values [0=False, 1=True]
    # TODO: implement power plant net
    pass

def set_probability(bn):
    # TODO: implement set probability function
    pass


In [ ]:
bn = make_power_plant_net()
bn = set_probability(bn)
vis.plot_bayes_net(bn)
vis.plot_all_cpt_tables(bn)


## Part 2: Exact Inference

In this part, you will compute probabilities using your Bayesian Network and exact methods.

You must implement:
- **Inference by Enumeration**
- **Inference by Variable Elimination**

You will also need **joint probability computation** when required so you must implement that too.

---

### Required Queries

Using both inference methods, compute:

- **Marginal probability the alarm sounds**  
  P(A = True)

- **Marginal probability the gauge reads hot**  
  P(G = True)

- **Posterior probability that the temperature is hot given evidence**  
  P(T = True ∣ A=True, Fa=False, Fg=False)

---

### What You Must Implement

✔ A function to compute **joint probability** using the whole network  
✔ `enumeration_ask(query_var, evidence, bn)`  
✔ `variable_elimination(query_var, evidence, bn)`  

For each query, print:
- Probability result using Enumeration
- Probability result using Variable Elimination
- Compare → Confirm that both methods match


In [ ]:
def joint_probability(bn, assignment):
    """
    Compute the joint probability P(X1, X2, ..., Xn) for a full assignment
    using the factorization of the Bayesian network.

    Parameters:
        bn : BayesNet
            The Bayesian network
        assignment : dict
            Dictionary mapping variable names to indices of their values
            in node.values, e.g., {'alarm': 1, 'temperature': 0}

    Returns:
        float : the joint probability of the assignment
    """
    # TODO: implement joint probability calculation
    pass



def enumeration_ask(query_var, evidence, bn):
    """
    Compute the probability distribution over query_var given evidence
    using enumeration (sum over all consistent assignments).

    Parameters:
        query_var : str
            The variable to query
        evidence : dict
            Dictionary of observed variable values {var_name: value}
        bn : BayesNet
            The Bayesian network

    Returns:
        dict : mapping of variable values to probabilities, e.g.,
               {0: 0.75, 1: 0.25}
    """
    # TODO: implement enumeration inference
    pass



def variable_elimination(query_var, evidence, bn):
    """
    Compute the probability distribution over query_var given evidence
    using the Variable Elimination algorithm.

    Parameters:
        query_var : str
            The variable to query
        evidence : dict
            Dictionary of observed variable values {var_name: value}
        bn : BayesNet
            The Bayesian network

    Returns:
        dict : mapping of variable values to probabilities, e.g.,
               {0: 0.75, 1: 0.25}
    """
    # TODO: implement variable elimination
    pass


In [ ]:
def enumeration_prob(query_var, value, evidence, bn):
    """
    Returns P(query_var=value | evidence) using enumeration.
    value: 0 or 1 (or whatever is in node.values)
    """
    dist = enumeration_ask(query_var, evidence, bn)
    return dist[value]


def variable_elimination_prob(query_var, value, evidence, bn):
    """
    Returns P(query_var=value | evidence) using variable elimination.
    value: 0 or 1 (or whatever is in node.values)
    """
    dist = variable_elimination(query_var, evidence, bn)
    return dist[value]

In [ ]:
queries = [
    ("alarm", 1, {}, "P(alarm=True)"),
    ("gauge", 1, {}, "P(gauge=True)"),
    ("temperature", 1, {"alarm": 1, "faulty alarm": 0, "faulty gauge": 0}, 
    "P(temperature=True | alarm=True, faulty alarm=False, faulty gauge=False)")
]

for q_var, target, ev, label in queries:
    print(f"\n--- {label} ---")
    enum_res = enumeration_prob(q_var, target, ev, bn)
    ve_res = variable_elimination_prob(q_var, target, ev, bn)
    print(f"Enumeration : {enum_res:.3f}")
    print(f"Var. Elim.  : {ve_res:.3f}")


## Part 3: Approximate Inference

Consider the following scenario: 

There are five frisbee teams (T1, T2, T3,...,T5). A match is played between teams Ti and Ti+1 to give a total of 5 matches, i.e. T1vsT2, T2vsT3, ..., T4vsT5, T5vsT1.

Each team can either win, lose, or draw in a match. Each team has a fixed but unknown skill level, represented as an integer from 0 to 3. Each match's outcome is probabilistically proportional to the difference in skill level between the teams.

We want to estimate the outcome of the last match (T5vsT1), given prior knowledge of other 4 matches. Rather than using inference, we will do so by sampling the network using three models: Rejection Sampling, Likelihood Weighting, and Gibbs Sampling.

First, work on a similar, smaller network! With just 3 teams. 

### 3.1) Build a smaller network with 3 teams

For the first sub-part, consider a smaller network with 3 teams : the Airheads, the Buffoons, and the Clods (A, B and C for short). 3 total matches are played. 
Build a Bayes Net to represent the three teams and their influences on the match outcomes. Assume the following variable conventions:

| variable name | description|
|---------|:------:|
|A| A's skill level|
|B | B's skill level|
|C | C's skill level|
|AvB | the outcome of A vs. B <br> (0 = A wins, 1 = B wins, 2 = tie)|
|BvC | the outcome of B vs. C <br> (0 = B wins, 1 = C wins, 2 = tie)|
|CvA | the outcome of C vs. A <br> (0 = C wins, 1 = A wins, 2 = tie)|

Assume that each team has the following prior distribution of skill levels:

|skill level|P(skill level)|
|----|:----:|
|0|0.15|
|1|0.45|
|2|0.30|
|3|0.10|

In addition, assume that the differences in skill levels correspond to the following probabilities of winning:

| skill difference <br> (T2 - T1) | T1 wins | T2 wins| Tie |
|------------|----------|---|:--------:|
|0|0.10|0.10|0.80|
|1|0.20|0.60|0.20|
|2|0.15|0.75|0.10|
|3|0.05|0.90|0.05|

In [ ]:
def make_game_network():
    # TODO: create a bayes net representation of the 3-team game problem
    pass


def set_probability(bn):
    # TODO: set the probabilities for the game network
    pass


In [ ]:
bn = make_game_network()
bn = set_probability(bn)
vis.plot_bayes_net(bn)
vis.plot_all_cpt_tables(bn)


### 3.2) Calculate posterior distribution for the 3rd match.

Suppose that you know the following outcome of two of the three games: A beats B and B draws with C. Start by calculating the posterior distribution for the outcome of the CvA match in calculate_posterior(). Use Enumeration ONLY. 

In [ ]:
def calculate_posterior(bn):
    # return posterior distribution for CvA
    pass

In [ ]:
posterior = calculate_posterior(bn)
posterior

### 3.3) Rejection Sampling

Now suppose you have 5 teams. You don't necessarily need to create a new network. You can just use the probability distributions tables from the previous part. Although be careful while indexing them.

Use **Rejection Sampling** to estimate the posterior probability of the 5th match (T5vT1). The algorithm works as follows:

1. Sample a full state (skills + matches) from the prior distribution.
2. Reject the sample if it is inconsistent with the evidence (the 4 known matches).
3. Keep track of how often each outcome occurs for the 5th match.
4. Normalize counts to get estimated probabilities.

Implement this in the function `rejection_sampling()`.


In [ ]:
def rejection_sampling(bayes_net, query_index, evidence, num_samples=10000, number_of_teams=5):
    """
    Estimate the posterior probability for a variable using Rejection Sampling.

    Parameters:
    - bayes_net: Bayesian network
    - query_index: index of variable to query (e.g., last match T5vT1)
    - evidence: dict {var_index: value} for observed variables
    - num_samples: total number of samples to generate
    - number_of_teams: number of teams participating in the competetion

    Returns:
    - posterior: list of probabilities for each possible value of query variable
    """
    # TODO: Implement rejection sampling
    posterior = [0.0] * 3  # 3 possible outcomes for a match
    return posterior


### 3.4) Likelihood Weighting

Next, use **Likelihood Weighting** to estimate the posterior probability of the 5th match (T5vT1). 

The algorithm works as follows:

1. Sample only the non-evidence variables from their conditional distributions.
2. Assign a weight to the sample based on the probability of the evidence given the sampled values.
3. Aggregate the weighted counts for the query variable.
4. Normalize the weighted counts to get estimated probabilities.

Implement this in the function `likelihood_weighting()`.


In [ ]:
def likelihood_weighting(bayes_net, query_index, evidence, num_samples=10000, number_of_teams=5):
    """
    Estimate the posterior probability for a variable using Likelihood Weighting.

    Parameters:
    - bayes_net: Bayesian network
    - query_index: index of variable to query (e.g., last match T5vT1)
    - evidence: dict {var_index: value} for observed variables
    - num_samples: total number of weighted samples to generate
    - number_of_teams: number of teams participating in the competetion


    Returns:
    - posterior: list of probabilities for each possible value of query variable
    """
    # TODO: Implement likelihood weighting
    posterior = [0.0] * 3
    return posterior


### 3.5) Gibbs Sampling

Next, use **Gibbs Sampling** to estimate the posterior probability of a variable (e.g., the 5th match, T5vT1).

The algorithm works as follows:

1. Initialize the state: Assign random values to all non-evidence variables; fix evidence variables to their known values.
2. Iteratively update variables: At each step, pick one non-evidence variable and sample a new value conditioned on its Markov blanket (its parents, children, and children's other parents).
3. Track the query variable: Record the value of the query variable at each iteration.
4. Repeat for many iterations: Continue updating variables to allow the chain to approach the stationary distribution.
5. Normalize the counts: After all iterations (optionally discarding an initial burn-in period), normalize the counts of the query variable’s values to estimate posterior probabilities.

Implement this in the function `gibbs_sampling()`.

In [ ]:
def gibbs_sampling(bayes_net, query_index, evidence, num_samples=10000, number_of_teams=5):
    """
    Gibbs sampling to estimate posterior for a query variable.

    Parameters:
    - bayes_net: Bayesian network
    - query_index: index of the variable to query
    - evidence: dict {var_index: value} for known evidence
    - num_samples: total Gibbs iterations
    - number_of_teams: default 5

    Returns:
    - posterior: list of probabilities for each value of the query variable
    """
    # TODO: Implement gibbs sampling
    posterior = [0.0] * 3
    return posterior
    
    

### 3.6) Compare Methods

Now that you have implemented sampling methods, you need to compare the prediction of these methods (approximate) with enumeration method (exact). First predict posterior probabilities for last match in a competetion of 3 teams with the same evidence as part 3.2 and then compare sampling results with enumeration.

After that predict posterior probabilities for last match in a competetion of 5 teams with the given evidence and show the difference between these methods.

Also test your sampling methods for other given evidence or number of teams and provide reason why your results correspond to the given evidence.

In [ ]:
# TODO: Compare approximate and exact methods for inference in 3 team version

In [ ]:
# TODO: Compare approximate methods for inference in 5 team version

In [ ]:
# TODO: Test your sampling methods for other given evidence or number of teams